<img src="https://drive.google.com/uc?export=view&id=1JJlC8pu04kkBtgnqdK8zTwYkUv0S2Fb0"  width="200" height="200">


# **XAIDA Toolbox for the Detection and Characterisation of Extreme Events**
## **Tutorial 2: Drought Detection using Long Short Term Memory Network**
### May 2023

---
## Tutorial workflow
---

## This notebook is designed to give users hands on experience of using the toolbox. Here, we implement a user defined deep learning archutecture as the syntax differs slightly to the PyOD outlier detection. We will alter the configuration file within the code to give users an intuition on how to alter parameters of the toolbox. Users can choose from 9 main archutectures from the [Pytorch Segmentation Models API Library](https://segmentation-modelspytorch.readthedocs.io/en/latest/). Alternatively one can create their own models within the models.py file, we refer to these as user defined models. For this tutorial we use the DroughtED database and a user defined LSTM to classify drought conditions using meteorological data. There are a number of exercises within each phase of the notebook where users will be asked to update the parameters to produce the best model.

    1. PREREQUISITES


    2. DATASET 
      
> Exercise 1 : Dataset Parameters
  
    3. MODEL
    
    4. TRAINING

> Exercise 2 : Setting Optimiser and Training Parameters

    5. EVALUATION AND VISUALIZATION

> Exercise 3 : Model Evaluation

---
## Prerequisites of the XAIDA Toolbox
---

Please, start running the following cells to check out the contents of the toolbox, install its dependencies and import the required libraries to make use of it.

#### a) Check out the contents XAIDA toolbox

In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')


In [2]:
# %cd /content/drive/MyDrive/XAIDA4Detection-Toolbox/notebooks


In [3]:
%ls -h

DroughtED_DeepLearning.ipynb  DroughtED_OutlierDetection.ipynb  imgs/


In [4]:
a = %pwd

if a.split("/")[-1] != "XAIDA":
    %cd ../
%ls -h

/home/miguelangelft/Documents/research/xaida/XAIDA4Detection
backbones/  evaluators/   main.py*                tutorials/     xaida_env/
configs/    experiments/  README.md               user_defined/
databases/  LICENSE       requirements_xaida.txt  utils/


#### b) Install dependencies

In [5]:
import sys
!{sys.executable} -m pip install -r requirements_xaida.txt

#### c)  Import required libraries

In [6]:
# GENERAL
import os, sys
from os.path import dirname, abspath

# NUMPY
import numpy as np

# PANDAS
import pandas as pd

# XARRAY
import xarray as xr

# ITERTOOLS
import itertools

# TIME, DATETIME
import time
from datetime import datetime

# SCIPY
from scipy.interpolate import interp1d

# SKLEARN
from sklearn.preprocessing import RobustScaler

# DATASET CLASSES
from databases import *

# MODEL TEMPLATE CLASS
from  backbones import *

# EVALUATION
from evaluators import *

# METRICS
from utils.misc import * 
from utils.setup_config import setup

import yaml
from os.path import dirname, abspath
from pathlib import Path
import sys

# MATPLOTLIB
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
pd.set_option('display.max_rows', 50)

/home/miguelangelft/Documents/research/xaida/XAIDA4Detection/xaida_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---

## 2: DATASET
---

#### 2.1 DroughtED

The DroughtED classification database consists of globally available meteorological features paired with location meta-data. It can be downloaded from [this link](https://www.kaggle.com/datasets/cdminix/us-drought-meteorological-data/download). 

- Data dimensionality:

Each entry is a daily drought level at a specific point in time in a specific US county, accompanied by 18 meteorological indicators.

- Drought levels:

DroughtED is a classification dataset with six levels of drought, depicted on the following image:

![](https://drive.google.com/uc?export=view&id=1DV-BjfcTV5e8iSSuQS93Hrt3mrRsakot)


One can also binarise the classes to make anomalous and non-anomalous classification:

![](https://drive.google.com/uc?export=view&id=1nGUn_VOttbepgMaG_t5Ccdhe32qP0jNr)

- Meteorological indicators:

The DroughtED database contains 18 meteorological indicators. 

 ||||
 |---|---|---|
 |- Minimum Wind Speed at 10 Meters (m/s) | - Specific Humidity at 2 Meters (g/kg) | - Temperature Range at 2 Meters (C)|
 |- Wind Speed at 10 Meters (m/s)|- Temperature at 2 Meters (C)|- Minimum Wind Speed at 50 Meters (m/s)|
 |- Maximum Temperature at 2 Meters (C)|- Wind Speed at 50 Meters (m/s)|- Earth Skin Temperature (C)|
 |- Wind Speed Range at 50 Meters (m/s)|- Maximum Wind Speed at 50 Meters (m/s)|- Maximum Wind Speed at 10 Meters (m/s)|
 |- Wind Speed Range at 10 Meters (m/s)|- Surface Pressure (kPa)|- Dew/Frost Point at 2 Meters (C)|
 |- Minimum Temperature at 2 Meters (C)|- Wet Bulb Temperature at 2 Meters (C)|- Precipitation (mm day-1)|

- Data subsets: For this tutorial, we will have available data from 2017 until 2020. 

## Configuration file
This is the most imporant file, for a set problem, this is the only file which requries editing. Here we read in the configuration file and create some folders to keep track of our experiments.

In [7]:
config_file = "config_DroughtED_LSTM.yaml"
config_path= Path('./').resolve() / 'configs' / config_file
experiment_ID= datetime.now().strftime('%H_%M_%S_%Y_%m_%d') +\
                '_jupyter_' + '_'.join(config_file.split('.')[0].split('_')[1:])
config= setup(config_path)

#We manually set the experiment ID
config['experiment_id']= experiment_ID
config['experiment_id']

'12_47_08_2023_05_26_jupyter_DroughtED_LSTM'

### The dataset file

For the user to be able to work with their own dataset, they have to create a dataset file NAME_database.py (DroughtED_database.py in this example) which will define a class NAME (DroughtED in this example). Within this class the data will be read, split and preprocessing carried out. Note that the Pytorch models use **different functions** to the outlier detection. Instead of returning the full dataset, this dataset class returns a sample of the dataset using the index \__getitem__ and the length of the dataset \__len__

The Dataset module stores the samples and their corresponding labels, while the DataLoader wraps an iterable around the Dataset to enable easy access to the samples. The pytorch models need the following dataset class structure:

![f](https://drive.google.com/uc?export=view&id=1yZJc3vuW-de8ctPr0uqy4e4RjK3QFAaA)


# EXERCISE 1

Hopefully you now have an intuition on how the toolbox works, which means it is your turn to implement your own data parameters within the configuration file. We have given some bounds and indicated the parameters in which it may be interesting to vary. The first choice you will need to make is how to binarise the data. Other parameters which might be interesting to change are the number of features, the time slices and the window size.

Below we show the **SYNTAX** to change the config variables within the
notebook, just to make it easier for you. This is not best practice and would be better to update the actual config file.

In [8]:
####### change the variables here ########
class_bound = 1 # Threshold for the binarisation 
number_of_features = 2 # number of features i.e. precip, temp etc..
features_selected = [0,3] # must list the location of the features
train_start = '2018-01-01'
train_end = '2018-12-31'
val_start = '2019-01-01'
val_end = '2019-12-31'
test_start = '2020-01-01'
test_end = '2020-12-31'
window_size = 30 # size of window to detect the event
batch_size = 64 


config['data']['class_bound'] = class_bound 
###
config['data']['input_size_train'] = number_of_features
config['data']['features_selected'] = features_selected 
###
config['data']['train_slice']['start'] = train_start
config['data']['train_slice']['end'] = train_end
###
config['data']['test_slice']['start'] = test_start
config['data']['test_slice']['end'] = test_end
###
config['data']['val_slice']['start'] = val_start
config['data']['val_slice']['end'] = val_end

config['data']['window_size'] = window_size

config['data']['batch_size'] = batch_size

Lets now read in the data classes with your chosen parameters:




In [9]:
data_train = eval(config['data']['name'])(config, period = 'train')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:12<00:00, 244.65it/s]


loaded 33739 samples


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:11<00:00, 281.65it/s]


loaded 33739 samples


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 91.42it/s]


[75.85287056522125, 24.147129434778744]


In [10]:
data_val = eval(config['data']['name'])(config, period = 'val')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:11<00:00, 279.85it/s]


loaded 33739 samples


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:11<00:00, 280.06it/s]


loaded 33739 samples


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 92.13it/s]

[86.1139927087347, 13.886007291265301]


In [11]:
data_test = eval(config['data']['name'])(config, period = 'test')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:11<00:00, 278.74it/s]


loaded 33863 samples


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:10<00:00, 284.59it/s]


loaded 33863 samples


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 86.08it/s]

[75.39202078965242, 24.607979210347576]


In [12]:
# Example data
print(data_test[1])

{'x': tensor([[-0.0892, -0.2065],
        [-0.1244, -0.0955],
        [ 1.0000,  0.0432],
        [-0.1312, -0.3965],
        [ 0.0217,  0.1654],
        [ 0.8218, -0.0083],
        [-0.1312, -0.2508],
        [-0.1312, -0.2170],
        [ 0.0117, -0.1006],
        [-0.0776,  0.0025],
        [-0.1312, -0.3965],
        [ 0.1577, -0.0140],
        [ 1.0000, -0.1299],
        [-0.1307, -0.2912],
        [-0.1312, -0.2390],
        [-0.1312, -0.1645],
        [ 0.4425, -0.0315],
        [-0.1312, -0.3965],
        [-0.1202, -0.1429],
        [-0.1312, -0.2883],
        [-0.1312, -0.2247],
        [-0.1312, -0.1089],
        [-0.1281, -0.0646],
        [ 0.7981,  0.0318],
        [-0.1312, -0.3965],
        [ 1.0000,  0.0477],
        [ 0.9731, -0.0455],
        [-0.1275, -0.1025],
        [-0.1165, -0.1757],
        [-0.1312, -0.0739]]), 'labels': tensor([0.])}


In [13]:
# Show one sample of data
print(data_test[:]['x'].shape)

torch.Size([33863, 30, 2])


In [14]:
# Show one sample of the ground-truth
print(data_test[:]['labels'])

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])


Load in the data using the default data loader. 

In [15]:
train_loader = DataLoader(data_train, shuffle=True,
                          batch_size = config['data']['batch_size'], 
                          num_workers = config['implementation']['data_loader']['num_workers'])
        
val_loader = DataLoader(data_val, 
                        batch_size = config['data']['batch_size'],
                        num_workers = config['implementation']['data_loader']['num_workers'])
        
test_loader = DataLoader(data_test, 
                        batch_size = 1,
                        num_workers = config['implementation']['data_loader']['num_workers'])

---
# 3. MODEL
---


Here we define the model architecture, we select user defined as we are using one of our own models: a Long Short Term Memory based network. When chosing a model from the Pytorch library, the parameters for the architecture can be found in the [Pytorch Segmentation Models API Documentation](https://segmentation-modelspytorch.readthedocs.io/en/latest/). The Pytorch Segmentation models have 9 models architectures for binary and multi class segmentation (including Unet), and hundreds of encoders.

The following image shows the minimum required parameters that this file should contain: 


![f](https://drive.google.com/uc?export=view&id=1uPo37M5qISe9-zEltGYsK1A_5ys5eEn6)

---
# 4. TRAINING 
---

Before training lets revisit the configuration file and take a look at the training parameters.




# EXERCISE 2

First, we are going to modify the optimizer used by our model. The parameters that we can tune are the following ones: 

*   Type of optimizer
*   Learning rate
*   Loss function and its parameters



Before training we need to create the model with the parameters specified in the configuration file. TemplateModel2D is the python object that will be created. It does not require any modification from the user.

In [16]:
##### Update config here ######
opt_type = "Adam" 
lr = 0.0001 # The rate at which the model learns 
loss_package = "torchvision.ops"
loss_type = "sigmoid_focal_loss" # binary_cross_entopy / sigmoid_focal_loss
alpha = 0.9
gamma = 4

config['implementation']['optimizer']['type'] = opt_type
config['implementation']['optimizer']['lr'] = lr
config['implementation']['loss']['package'] = loss_package
config['implementation']['loss']['type'] = loss_type
config['implementation']['loss']['params']['alpha'] = alpha
config['implementation']['loss']['params']['gamma'] = gamma

Now we should decide how long we want the model to train by selecting the number of epochs.

In [17]:
##### Update config here ######
epoch = 100 # number of epochs the model will train

config['implementation']['trainer']['epochs'] = epoch

---
## 4.1 TRAINING LOOP AND EXECUTION
---

In [18]:
#Create experimental folder structure if it was not already created
if not Path(config['save_path']).name == config['experiment_id'].replace('/', ''):
    save_path= Path(config['save_path']) / Path(config['experiment_id'])
    config['save_path']= str(save_path.resolve())
    save_path.mkdir(exist_ok=True, parents=True) 
else:
    save_path= Path(config['save_path'])
print(f'Logging experiment data at: {save_path}')

classifier = PytorchBackbone(config)
# Load data
classifier.load_data()
# Model
classifier.implement_model()
classifier.model

Logging experiment data at: experiments/12_47_08_2023_05_26_jupyter_DroughtED_LSTM


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:11<00:00, 269.85it/s]


loaded 33739 samples


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3108/3108 [00:11<00:00, 278.08it/s]


loaded 33739 samples


...

loaded 33863 samples


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 73.75it/s]

[75.39202078965242, 24.607979210347576]



Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


PytorchModel(
  (model): UD_LSTM(
    (lstm): LSTM(2, 100, batch_first=True)
    (fc): Linear(in_features=100, out_features=1, bias=True)
  )
  (loss): Loss()
)

Model Summary

Next we start training our model.

In [19]:
model= classifier.train()

Missing logger folder: /home/miguelangelft/Documents/research/xaida/XAIDA4Detection/experiments/12_47_08_2023_05_26_jupyter_DroughtED_LSTM/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type    | Params
---------------------------------------
0 | model      | UD_LSTM | 41.7 K
1 | model.lstm | LSTM    | 41.6 K
2 | model.fc   | Linear  | 101   
3 | loss       | Loss    | 0     
---------------------------------------
41.7 K    Trainable params
0         Non-trainable params
41.7 K    Total params
0.167     Total estimated model params size (MB)


Epoch 0:  50%|██████████████████████████████████████████████████████████████████▌                                                                  | 264/528 [00:03<00:03, 83.05it/s, loss=0.0103, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 0:  57%|███████████████████████████████████████████████████████████████████████████▌                                                         | 300/528 [00:04<00:03, 68.54it/s, loss=0.0103, v_num=0]


Epoch 0:  64%|█████████████████████████████████████████████████████████████████████████████████████▍                                               | 339/528 [00:04<00:02, 73.12it/s, loss=0.0103, v_num=0]


Epoch 0:  72%|███████████████████████████████████████████████████████████████████████████████████████████████▏                                     | 378/528 [00:04<00:01, 77.32it/s, loss=0.0103, v_num=0]


Epoch 0:  79%|█████████████████████████████████████████████████████████████████████████████████████████████████████████                            | 417/528 [00:05<00:01, 81.24it/s, loss=0.0103, v_num=0]


...

Epoch 12:  78%|█████████████████████████████████████████████████████████████████████████████████████████████████████▉                             | 411/528 [00:05<00:01, 73.51it/s, loss=0.00926, v_num=0]


Epoch 12:  85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                   | 450/528 [00:05<00:01, 76.35it/s, loss=0.00926, v_num=0]


Epoch 12:  93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎         | 489/528 [00:06<00:00, 79.56it/s, loss=0.00926, v_num=0]


Epoch 12: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [00:06<00:00, 82.35it/s, loss=0.00926, v_num=0]


Epoch 12: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [00:06<00:00, 82.12it/s, loss=0.00926, v_num=0]


---
# 5. EVALUATION AND VISUALISATION OF RESULTS
---

Here we show two 1D evaluators in the form of the confusion matrix and the temporal evolution.

In [ ]:
classifier.test()
output_data= classifier.inference()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33863/33863 [02:36<00:00, 216.71it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_Accuracy                 0.0
     test_CohenKappa                0.0
      test_F1Score                  0.0
     test_Precision                 0.0
       test_Recall                  0.0
     test_loss_epoch        0.00871817022562027
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Performing visualization:   7%|█████████▉                                                                                                                           | 2526/33863 [00:01<00:18, 1723.18it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Performing visualization:  21%|████████████████████████████▎                                                                                                        | 7200/33863 [00:04<00:17, 1559.31it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000